In [1]:
!pip install transformers

     |████████████████████████████████| 4.0 MB 5.3 MB/s 
     |████████████████████████████████| 77 kB 2.7 MB/s 
     |████████████████████████████████| 596 kB 19.7 MB/s 
     |████████████████████████████████| 895 kB 30.7 MB/s 
     |████████████████████████████████| 6.6 MB 28.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
import nltk
!pip install wordninja
!pip install --upgrade gensim
nltk.download('stopwords')
nltk.download('wordnet')

     |████████████████████████████████| 541 kB 4.3 MB/s 
  Created wheel for wordninja: filename=wordninja-2.0.0-py3-none-any.whl size=541551 sha256=e35862021391231cb4f853a518f45dcac8e0372b567e270d9aead29fd6c20013
  Stored in directory: /root/.cache/pip/wheels/dd/3f/eb/a2692e3d2b9deb1487b09ba4967dd6920bd5032bfd9ff7acfc
Successfully built wordninja
     |████████████████████████████████| 24.1 MB 7.5 kB/s 
  Attempting uninstall: gensim
    Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [12]:
from google.colab import drive
import pandas as pd 
from sklearn.metrics import confusion_matrix,accuracy_score,roc_auc_score,roc_curve,auc
import nltk
from sklearn import feature_extraction, model_selection, naive_bayes, pipeline, manifold, preprocessing
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble

import pandas, xgboost, numpy, textblob, string
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers
from gensim.models import Word2Vec
import numpy as np


In [4]:
drive.mount('/content/gdrive')

Mounted at /content/gdrive



### Improve data
---



In [ ]:
data=pd.read_csv('gdrive/My Drive/thesis/predictionAssociationo_Inheritance/DatacONCEPTS_Clean.csv')

In [ ]:
data

,assocs,packageName,Source,Target
0,inherits from,elements,MultiLineInputElement,ScoutingElement
1,inherits from,elements,NumberDropDownElement,ScoutingElement
2,inherits from,elements,SingleLineInputElement,ScoutingElement
3,inherits from,elements,WeightingSliderElement,ScoutingElement
4,inherits from,h2,CommentTable,Table
...,...,...,...,...
196942,mVLCPLayerParser,mplayer,VLCPlayer,VLCPlayerParser
196943,mVLCPLayerErrorParser,mplayer,VLCPlayer,VLCPlayerErrorParser
196944,mPlayerProperties,mplayer,VLCPlayer,PlayerProperties
196945,_properties,mplayer,VLCPlayerErrorParser,PlayerProperties


In [ ]:
def changeAssociation(assoc):
  if assoc!= 'inherits from':
    return 'association'
  return 'inherits from'
df=data
df['assocs_'] = df.apply(lambda row: changeAssociation(row['assocs']),axis = 1)
df=df.dropna().reset_index()
df

,index,assocs,packageName,Source,Target,assocs_
0,0,inherits from,elements,MultiLineInputElement,ScoutingElement,inherits from
1,1,inherits from,elements,NumberDropDownElement,ScoutingElement,inherits from
2,2,inherits from,elements,SingleLineInputElement,ScoutingElement,inherits from
3,3,inherits from,elements,WeightingSliderElement,ScoutingElement,inherits from
4,4,inherits from,h2,CommentTable,Table,inherits from
...,...,...,...,...,...,...
196421,196942,mVLCPLayerParser,mplayer,VLCPlayer,VLCPlayerParser,association
196422,196943,mVLCPLayerErrorParser,mplayer,VLCPlayer,VLCPlayerErrorParser,association
196423,196944,mPlayerProperties,mplayer,VLCPlayer,PlayerProperties,association
196424,196945,_properties,mplayer,VLCPlayerErrorParser,PlayerProperties,association


In [ ]:
df2=df.head(5)

In [ ]:
data= pd.DataFrame([])

data['Source']=df['Source']

data['Target']=df['Target']
data['Type']=df['assocs_']
data

,Source,Target,Type
0,MultiLineInputElement,ScoutingElement,inherits from
1,NumberDropDownElement,ScoutingElement,inherits from
2,SingleLineInputElement,ScoutingElement,inherits from
3,WeightingSliderElement,ScoutingElement,inherits from
4,CommentTable,Table,inherits from
...,...,...,...
196421,VLCPlayer,VLCPlayerParser,association
196422,VLCPlayer,VLCPlayerErrorParser,association
196423,VLCPlayer,PlayerProperties,association
196424,VLCPlayerErrorParser,PlayerProperties,association


In [ ]:
#data.to_csv('gdrive/My Drive/thesis/predictionAssociationo_Inheritance/processedDataV1.csv')

In [ ]:
data=pd.read_csv('gdrive/My Drive/thesis/predictionAssociationo_Inheritance/processedDataV1.csv')
del data['Unnamed: 0']

In [ ]:
data

,Source,Target,Type
0,MultiLineInputElement,ScoutingElement,inherits from
1,NumberDropDownElement,ScoutingElement,inherits from
2,SingleLineInputElement,ScoutingElement,inherits from
3,WeightingSliderElement,ScoutingElement,inherits from
4,CommentTable,Table,inherits from
...,...,...,...
196421,VLCPlayer,VLCPlayerParser,association
196422,VLCPlayer,VLCPlayerErrorParser,association
196423,VLCPlayer,PlayerProperties,association
196424,VLCPlayerErrorParser,PlayerProperties,association


In [ ]:
data=data[data.Source.isin(['main']) == False]
data=data.reset_index()



In [ ]:
del data['index']

In [ ]:
data.Source.value_counts().head(50)

In [ ]:
df3=data

In [ ]:
import wordninja

###Remove By 

In [ ]:
import string
def removeAlphab(input):
  res=''
  x= wordninja.split(input)
  for i in x: 
    
    if i.lower() not in ['by']: 
      res=res+i
    else: 
      print(input)


  return res

In [ ]:
df3['cleanSource']=df3.apply(lambda row : removeAlphab(row['Source']), axis=1)
df3['cleanTarget']=df3.apply(lambda row : removeAlphab(row['Target']), axis=1)

UsedBy
UsedBy
RetweetByMe
RetweetByUser
HeuristicByPermutation
HeuristicByPermutation
CircleByCenterPoint
GroupByActionmpl
GroupByPackagempl
FragmentByIdProcessor
FragmentByTagProcessor
ViewByIdProcessor
FragmentByIdValidator
FragmentByTagValidator
ViewByIdValidator
ReferredBy
OrderByExpr
OrderByStable
BySeq
CancelBetsByMarketRe
CancelBetsByMarketRe
CancelBetsByMarketResp
CancelBetsByMarketResp
ResourceByPropertyMultiCategorize
ResourceByUriMultiCategorize
ResourceByUriTypeCategorize
ResourceByUriTypeMultiCategorize
ResourceByUriTypeMultiCategorize
SearchByTwistor
OrderByCriterion
FitByMeanStatistics
FitByMedianStatistics
RankingByCount
RequestingAnAccountBalanceByValidAccountIdButAnInvalidAccountBalanceId
RequestingAnAccountBalanceByAnInvalidAccountId
ByClassName
ByCssSelector
ById
ByLinkText
ByName
ByPartialLinkText
ByTagName
GroupByFactory
ChangeSizeByBrick
ChangeVolumeByBrick
ChangeByBrick
KomplexbyIkea
PayByBinding
SybylAtomTypeMatcher
SybylAtomTypeMatcher
SybyDescriptorFormat
NPC

In [ ]:
df3

,Source,Target,Type,cleanSource,cleanTarget
0,MultiLineInputElement,ScoutingElement,inherits from,MultiLineInputElement,ScoutingElement
1,NumberDropDownElement,ScoutingElement,inherits from,NumberDropDownElement,ScoutingElement
2,SingleLineInputElement,ScoutingElement,inherits from,SingleLineInputElement,ScoutingElement
3,WeightingSliderElement,ScoutingElement,inherits from,WeightingSliderElement,ScoutingElement
4,CommentTable,Table,inherits from,CommentTable,Table
...,...,...,...,...,...
196411,VLCPlayer,VLCPlayerParser,association,VLCPlayer,VLCPlayerParser
196412,VLCPlayer,VLCPlayerErrorParser,association,VLCPlayer,VLCPlayerErrorParser
196413,VLCPlayer,PlayerProperties,association,VLCPlayer,PlayerProperties
196414,VLCPlayerErrorParser,PlayerProperties,association,VLCPlayerErrorParser,PlayerProperties


In [ ]:

#df3.to_csv('gdrive/My Drive/thesis/predictionAssociationo_Inheritance/processedDataV2.csv')

##Training Model

### combined features (preprocessing)

In [ ]:
dataframe = pd.DataFrame([], columns = ['sequence', 'Type'])
seq=[]
for i, row in df3.iterrows():
  seq='('+ row['cleanSource'] +' , '+ row['cleanTarget'] +')'
  dataframe=dataframe.append({'sequence': seq, 'Type': row['Type']}, ignore_index=True)
      

dataframe

In [ ]:
dataframe.to_csv('gdrive/My Drive/thesis/predictionAssociationo_Inheritance/processedDataV3.csv')

In [ ]:
import re
def utils_preprocess_text(text, flg_stemm=False, flg_lemm=True, lst_stopwords=None):
    ## clean (convert to lowercase and remove punctuations and     characters and then strip)
    text = re.sub(r'[^\w\s]', '', str(text).lower().strip())
            
    ## Tokenize (convert from string to list)
    lst_text = text.split()
    ## remove Stopwords
    if lst_stopwords is not None:
        lst_text = [word for word in lst_text if word not in 
                    lst_stopwords]
                
    ## Stemming (remove -ing, -ly, ...)
    if flg_stemm == True:
        ps = nltk.stem.porter.PorterStemmer()
        lst_text = [ps.stem(word) for word in lst_text]
                
    ## Lemmatisation (convert the word into root word)
    if flg_lemm == True:
        lem = nltk.stem.wordnet.WordNetLemmatizer()
        lst_text = [lem.lemmatize(word) for word in lst_text]
            
    ## back to string from list
    text = " ".join(lst_text)
    return text

In [ ]:
dataframe["text_clean"] = dataframe["sequence"].apply(lambda x: 
          utils_preprocess_text(x, flg_stemm=False, flg_lemm=True,  lst_stopwords=lst_stopwords))
dataframe.head()

In [ ]:
#dataframe.to_csv('gdrive/My Drive/thesis/predictionAssociationo_Inheritance/CleanForPrediction.csv')

In [ ]:
data=pd.read_csv('gdrive/My Drive/thesis/predictionAssociationo_Inheritance/processedDataV2.csv')

### word embedding for  classification

In [5]:
dtf=pd.read_csv('gdrive/My Drive/thesis/predictionAssociationo_Inheritance/CleanForPrediction.csv')

In [6]:
dtf=dtf.dropna()
dtf=dtf.reset_index()
del dtf['Unnamed: 0']
del dtf['index']

In [ ]:
dtf

In [7]:
from keras.layers.embeddings import Embedding

In [8]:
from sklearn.model_selection import train_test_split
df_train, df_test = train_test_split(dtf, test_size=0.2, random_state=42)
print("TRAIN size:", len(df_train))


print("TEST size:", len(df_test))

TRAIN size: 157122
TEST size: 39281


###Target

NOooooooooooooooooooooooooooooooo

In [38]:
inverse_dic

{'association': 0, 'inherits from': 1}

In [13]:

dic_y_mapping = {n:label for n,label in 
                 enumerate(np.unique(df_train.Type))}
inverse_dic = {v:k for k,v in dic_y_mapping.items()}
Y_train = np.array([inverse_dic[y] for y in df_train.Type])

Y_test= np.array([inverse_dic[y] for y in df_test.Type])

In [14]:

lst_stopwords = nltk.corpus.stopwords.words("english")


###pretrained

In [15]:
import gensim

In [ ]:
documents = [_text.split() for _text in dtf.text_clean]
documents

In [17]:

w2vModel = gensim.models.word2vec.Word2Vec(sentences =documents,min_count=5, workers=8, vector_size=300)
w2vModel.build_vocab(documents)
w2vModel.wv.vectors_lockf = np.ones(len(w2vModel.wv))


In [18]:
w2vModel.wv.intersect_word2vec_format('gdrive/My Drive/thesis/embeded/pretrained/GoogleNews-vectors-negative300.bin', binary=True,lockf=1)

In [19]:
w2vModel.train(documents, total_examples=len(documents), epochs=3)

(647536, 1175937)

In [20]:
w2vModel.save('gdrive/My Drive/thesis/embeded/pretrained/trainedTest')

In [36]:
model = Word2Vec.load('gdrive/My Drive/thesis/embeded/pretrained/trainedTest')

In [66]:
'''embeddings = np.zeros((len(documents)+1, 300))
for word,idx in w2vModel.wv.key_to_index.items():
    
    ## update the row with vector
    try:
        
        embeddings[idx] =  w2vModel.wv[word]
    ## if word not in model then skip and the row stays all 0s
    except:
        print(word)
        pass'''

In [21]:
result = w2vModel.wv.most_similar(positive=['car'], topn=10)
result

[('vehicle', 0.7821096181869507),
 ('truck', 0.6735789775848389),
 ('driver', 0.5962077379226685),
 ('pickup', 0.48767825961112976),
 ('motor', 0.4871765375137329),
 ('bus', 0.4693371057510376),
 ('wheel', 0.46299615502357483),
 ('rentalcar', 0.4341348111629486),
 ('convertable', 0.41912972927093506),
 ('house', 0.4134519100189209)]

In [ ]:
'''def attention_layer(inputs, neurons):
    x = layers.Permute((2,1))(inputs)
    x = layers.Dense(neurons, activation="softmax")(x)
    x = layers.Permute((2,1), name="attention")(x)
    x = layers.multiply([inputs, x])
    return x

## input
x_in = layers.Input(shape=(15,))
## embedding
x = layers.Embedding(input_dim=embeddings.shape[0],  
                     output_dim=embeddings.shape[1], 
                     weights=[embeddings],
                     input_length=15, trainable=False)(x_in)
## apply attention
x = attention_layer(x, neurons=15)
## 2 layers of bidirectional lstm
x = layers.Bidirectional(layers.LSTM(units=15, dropout=0.2, 
                         return_sequences=True))(x)
x = layers.Bidirectional(layers.LSTM(units=15, dropout=0.2))(x)
## final dense layers
x = layers.Dense(64, activation='relu')(x)
y_out = layers.Dense(3, activation='softmax')(x)
## compile
model = models.Model(x_in, y_out)
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam', metrics=['accuracy'])

model.summary()''''

In [ ]:
'''dic_y_mapping = {n:label for n,label in 
                 enumerate(np.unique(df_train.Type))}
inverse_dic = {v:k for k,v in dic_y_mapping.items()}
y_train = np.array([inverse_dic[y] for y in df_train.Type])'''
## train
'''training = model.fit(x=, y=y_train, batch_size=256, 
                     epochs=10, shuffle=True, verbose=0, validation_split=0.3)


In [22]:
class Word2VecVectorizer:
  def __init__(self, model):
    print("Loading in word vectors...")
    self.word_vectors = model
    print("Finished loading in word vectors")

  def fit(self, data):
    pass

  def transform(self, data):
    # determine the dimensionality of vectors
    v = self.word_vectors.get_vector('king')
    self.D = v.shape[0]

    X = np.zeros((len(data), self.D))
    n = 0
    emptycount = 0
    for sentence in data:
      tokens = sentence.split()
      vecs = []
      m = 0
      for word in tokens:
        try:
          # throws KeyError if word not found
          vec = self.word_vectors.get_vector(word)
          vecs.append(vec)
          m += 1
        except KeyError:
          pass
      if len(vecs) > 0:
        vecs = np.array(vecs)
        X[n] = vecs.mean(axis=0)
      else:
        emptycount += 1
      n += 1
    print("Numer of samples with no words found: %s / %s" % (emptycount, len(data)))
    return X


  def fit_transform(self, data):
    self.fit(data)
    return self.transform(data)

In [23]:
vectorizer = Word2VecVectorizer(w2vModel.wv)

Loading in word vectors...
Finished loading in word vectors


In [24]:
Xtrain = vectorizer.fit_transform(df_train.text_clean)
Xtest = vectorizer.transform(df_test.text_clean)


Numer of samples with no words found: 35064 / 157122
Numer of samples with no words found: 8803 / 39281


In [31]:
#w2vModel.wv.save_word2vec_format("gdrive/My Drive/thesis/embeded/pretrained/vectorizer")

###Results

In [25]:
from sklearn.ensemble import RandomForestClassifier

# create the model, train it, print scores
clf = RandomForestClassifier(n_estimators=200)

clf.fit(Xtrain, Y_train)


RandomForestClassifier(n_estimators=200)

In [26]:
y_pred = clf.predict(Xtest)
print(metrics.classification_report(Y_test, y_pred,  digits=5))



              precision    recall  f1-score   support

           0    0.81624   0.64410   0.72003     17241
           1    0.76102   0.88657   0.81901     22040

    accuracy                        0.78015     39281
   macro avg    0.78863   0.76534   0.76952     39281
weighted avg    0.78526   0.78015   0.77557     39281



In [27]:
print("train score:", clf.score(Xtrain, Y_train))

train score: 0.8351599394101399


In [28]:
print("test score:", clf.score(Xtest, Y_test))

test score: 0.7801481632341335


In [29]:
import pickle
pickle.dump(clf, open("gdrive/My Drive/thesis/predictionAssociationo_Inheritance/RandomForest_model2.sav", 'wb'))

In [33]:
loaded_model = pickle.load(open("gdrive/My Drive/thesis/predictionAssociationo_Inheritance/RandomForest_model2.sav", 'rb'))
result = loaded_model.score(Xtest, Y_test)
print(result)

0.7801481632341335


In [61]:
test='(employee, company)'

In [47]:
import re
import nltk
def utils_preprocess_text(text, flg_stemm=False, flg_lemm=True, lst_stopwords=None):
    ## clean (convert to lowercase and remove punctuations and     characters and then strip)
    text = re.sub(r'[^\w\s]', '', str(text).lower().strip())
            
    ## Tokenize (convert from string to list)
    lst_text = text.split()
    ## remove Stopwords
    if lst_stopwords is not None:
        lst_text = [word for word in lst_text if word not in 
                    lst_stopwords]
                
    ## Stemming (remove -ing, -ly, ...)
    if flg_stemm == True:
        ps = nltk.stem.porter.PorterStemmer()
        lst_text = [ps.stem(word) for word in lst_text]
                
    ## Lemmatisation (convert the word into root word)
    if flg_lemm == True:
        lem = nltk.stem.wordnet.WordNetLemmatizer()
        lst_text = [lem.lemmatize(word) for word in lst_text]
            
    ## back to string from list
    text = " ".join(lst_text)
    return text

In [62]:
test2=utils_preprocess_text(test)
test2

'employee company'

In [39]:
inverse_dic

{'association': 0, 'inherits from': 1}

In [63]:
res=vectorizer.transform([test2])
print(len(res[0]))
loaded_model.predict(res)

Numer of samples with no words found: 0 / 1
300


array([0])